In [ ]:
# ETL Pipeline Santander DIO

## Importando o csv com os ids dos clientes

import pandas as pd

df = pd.read_csv('clients-id.csv')

print(df)

users_id = df['User ID'].tolist()

print(users_id)

## Pegando os clientes da API da Santander Dev Week

site_api_url = 'https://sdw-2023-prd.up.railway.app'

import requests
import json

def get_user(id):
    response = requests.get(f'{site_api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None

users = [user for id in users_id if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

## Usando LangChain & HuggingFace Hub Api para gerar marketing personalizado para cada usuário

hug_api_key='YOUR_HUGGINGFACE_API_KEY'

!pip -q install langchain huggingface_hub transformers sentence_transformers accelerate bitsandbytes

import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hug_api_key

from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """You are a {profession}.

Explain to me the importance of investments!"""

prompt = PromptTemplate(template=template, input_variables=["profession"])

llm_chain = LLMChain(prompt=prompt,
                      llm=HuggingFaceHub(repo_id="gpt2",
                                         model_kwargs={"temperature":0.9,
                                                       "max_length":100}),
                    verbose=True)

#question = "How important is to maintain continuous investment in stock shares?"

print(llm_chain.run("Investment Specialist"))
print(llm_chain.run("Market analyst"))
print(llm_chain.run("Investment Analyst"))

## Traduzir para Português

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("unicamp-dl/translation-en-pt-t5")

model = AutoModelForSeq2SeqLM.from_pretrained("unicamp-dl/translation-en-pt-t5")

enpt_pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

ans = enpt_pipeline("translate English to Portuguese: I like to eat rice.")

first_key, first_value = next(iter(ans[0].items()))

print(first_value)

## Adicionando a mensagem personalizada

import random

print(random.randint(1,10))

roles = [
    "Investment banker",
    "Underwriter",
    "Broker",
    "Investment analyst",
    "Trader",
    "Portfolio manager",
    "Hedge fund analyst",
    "Finance manager"
]

for user in users:
    news = enpt_pipeline(f'translate English to Portuguese:{llm_chain.run(roles[random.randint(0,len(roles)-1)])}')
    first_k, first_v = next(iter(news[0].items()))
    print(first_v)
    user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": first_v
  })

## Fazendo put nos usuários já atualizados

print(json.dumps(users, indent=2))

def update_user(user):
  response = requests.put(f"{site_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

